In [6]:
import os
import sys
import subprocess
import xml.etree.ElementTree as ET

In [7]:
class SumoFiles:
    """
    A class for working with SUMO files.

    Methods:
        save_net_file(osm_file, output_directory=None):
            Saves a SUMO network file from an OpenStreetMap file.
        save_rou_file(route_data, file_name):
            Saves a SUMO routes file from a list of routes.
    """
    def __init__(self):
        """
        Initializes a new SumoFiles object.
        """
        try:
            self._add_path_sumo()
        except Exception as error:
            print("Error in constructor")
            logging.error(error)

    def _add_path_sumo(self):
        """
        Adds the SUMO tools path to the system path.

        Raises:
            Exception: If the SUMO_HOME environment variable is not set or the SUMO tools path does not exist.
        """
        path_sumo_tools = os.path.join(os.environ["SUMO_HOME"], "tools")
        if not os.path.exists(path_sumo_tools):
            try:
                sys.path.append(path_sumo_tools)
            except Exception as error:
                print(
                    "SUMO_HOME is not an environment variable. Function _add_path_sumo"
                )
                raise error

    @staticmethod
    def save_net_file(osm_file, output_directory=None):
        """
        Saves a SUMO network file from an OpenStreetMap file.

        Args:
            osm_file (str): The path to the OpenStreetMap file.
            output_directory (str): The path to the output directory. If None, the network file is saved in the same directory as the OpenStreetMap file.

        Returns:
            str: The path to the saved network file.

        Raises:
            Exception: If the osmBuild module is not installed or the network file creation fails.
        """
        try:
            import osmBuild

            prefix = osm_file.replace(".osm.xml", "")
            args = [
                "--osm-file",
                f"{osm_file}",
                "--vehicle-classes=road",
                f"--prefix={prefix}",
            ]
            if output_directory:
                args = [
                    "--osm-file",
                    f"{osm_file}",
                    "--vehicle-classes=road",
                    f"--prefix={prefix}",
                    f"--output-directory={output_directory}",
                ]
            osmBuild.build(args)
            _file_net_name = prefix + ".net.xml"
            return _file_net_name
        except Exception as error:
            print("Sumo dictionary creation failed. Function save_net_file")
            raise error

    @staticmethod
    def save_rou_file(vehicle_types, vehicles, file_name):
        """
        Saves the specified vehicle types and vehicles to a SUMO routes file.

        Args:
            vehicle_types (List[VehicleType]): The vehicle types to save to the file.
            vehicles (List[Vehicle]): The vehicles to save to the file.
            file_name (str): The name of the file to save the data to.

        Returns:
            bool: True if the file was saved successfully, False otherwise.
        """
        root = ET.Element(
            "routes",
            {
                "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
                "xsi:noNamespaceSchemaLocation": "http://sumo.dlr.de/xsd/routes_file.xsd",
            },
        )
        
        for veh_type in vehicle_types:
            veh_type_keys = veh_type.get_keys()
            vtype = ET.SubElement(root, "vType")
            for key in veh_type_keys:
                vtype.set(key, str(veh_type.get_attribute(key)))

        for vehicle in vehicles:
            vehicle_keys = vehicle.get_keys()
            veh_xml = ET.SubElement(root, "vehicle")
            for key in vehicle_keys:
                if key == "route":
                    route = ET.SubElement(veh_xml, "route")
                    veh_route = vehicle.get_attribute(key)
                    veh_route_sumo = veh_route.route
                    route.set("edges", " ".join(map(str, veh_route_sumo)))
                else:
                    veh_xml.set(key, str(vehicle.get_attribute(key)))
                
        tree = ET.ElementTree(root)
        tree.write(file_name, xml_declaration=True)
        return True

In [8]:
!jupyter nbconvert --to python sumo_files/notebooks/sumo_files.ipynb

[NbConvertApp] Converting notebook sumo_files/notebooks/sumo_files.ipynb to python
[NbConvertApp] Writing 4494 bytes to sumo_files\notebooks\sumo_files.py
